#  Code Authored by Avik Chakraborty. {-}
# This program focuses mainly on Data Preparation of Time-Series Data. {-}
# The dataset is a flight ticket price. {-}

# Importing the necesary libraries.

In [1]:
import pandas
import seaborn
import matplotlib.pyplot as mplot

In [ ]:
trainDataset = pandas.read_excel('Train Set.xlsx')
trainDataset.head()

In [ ]:
testDataset = pandas.read_excel('Test Set.xlsx')
testDataset.head()

# Data Description.
##### Airline : name of the airline.
##### Date_of_Journey : start date of journey.
##### Source : name of the city from where the airplane is departing.
##### Destination : name of the city from where the airplane will arrive.
##### Route : defines the route of the airplane.
##### Dep_Time : departure time.
##### Arrival_Time : arrival time.
##### Duration : Duration of the flight.
##### Total_Stops : number of places the airplane will stop between Source and Destination.
##### Additional_Info : describes some additional info for the airplane.
##### Price : price of the airplane.

In [ ]:
trainDataset.info()

In [ ]:
testDataset.info()

In [ ]:
trainDataset.describe()

In [ ]:
testDataset.describe()

In [ ]:
trainDataset.shape

In [ ]:
testDataset.shape

# Data Pre-processing.
## Data Cleaning.
### Checking for NAN values.
#### Training Set.

In [ ]:
trainDataset.isna().sum()

In [ ]:
trainDataset[(trainDataset['Route'].isna() == True) | (trainDataset['Total_Stops'].isna() == True)]

#### Since there is only 1 NAN value in train dataset, removing it.

In [12]:
trainDataset = trainDataset.drop(index = 9039, axis = 0)

In [ ]:
trainDataset.isna().sum()

##### Testing Set.

In [ ]:
testDataset.isna().sum()

##### There is no NaN values in test dataset.
### Checking for outliers.

In [ ]:
seaborn.boxplot(data = trainDataset, x = 'Price')
mplot.show()

In [3]:
from sklearn.ensemble import IsolationForest
import numpy as np

# Example multivariate data
data = np.array([[10, 2], [12, 3], [11, 2.5], [14, 4], [13, 3.5], [17, 5], [12, 2.8], [10, 2.2], [100, 20], [11, 2.3]])

# Train the Isolation Forest model
clf = IsolationForest(contamination=0.2)  # `contamination` specifies the proportion of outliers
clf.fit(data)

# Predict outliers (-1 indicates outliers)
predictions = clf.predict(data)
outlier_positions = np.where(predictions == -1)[0]
print("Outlier positions:", outlier_positions)

Outlier positions: [5 8]


In [4]:
data[outlier_positions]

array([[ 17.,   5.],
       [100.,  20.]])

## Data Preparation.
### Dealing with Airline column.
##### The unique values in the airline column is mapped with each airline representing a value. FOr e.g.: IndiGo = 1, Air India = 2.

In [ ]:
trainDataset.head()

In [ ]:
trainDataset['Airline'].unique()

In [ ]:
testDataset['Airline'].unique()

In [ ]:
len(trainDataset['Airline'].unique())

In [ ]:
len(testDataset['Airline'].unique())

The unique values in the test set is same as the one in train set, therefore considering the unique values in train set.

In [ ]:
uniqueAirlines = [airline for airline in trainDataset['Airline'].unique()]
uniqueAirlines

In [ ]:
count = 1
temp = {}

for airline in uniqueAirlines:
    temp.update({airline : count})
    count += 1

uniqueAirlines = temp
del temp
uniqueAirlines

#### For training Set.

In [ ]:
trainDataset['Airline'] = trainDataset['Airline'].map(uniqueAirlines)
trainDataset.head()

#### For Testing Set.

In [ ]:
testDataset['Airline'] = testDataset['Airline'].map(uniqueAirlines)
testDataset.head()

### Dealing with Date_of_Journey column in training set.
##### a) Formatting the date into a datetime format.

In [ ]:
trainDataset['Date_of_Journey'] = trainDataset['Date_of_Journey'].astype(dtype = 'datetime64[us]')
trainDataset.head()

##### b) Dividing the date in Date_of_Journey dataset into date, month and year.

In [ ]:
trainDataset['Journey_start_Date'] = trainDataset['Date_of_Journey'].dt.day
trainDataset['Journey_start_Month'] = trainDataset['Date_of_Journey'].dt.month
trainDataset['Journey_start_Year'] = trainDataset['Date_of_Journey'].dt.year
trainDataset = trainDataset.drop(columns = ['Date_of_Journey'], axis = 1)
trainDataset.head()

##### c) Shifting the rows from the end to the respective position.

In [ ]:
trainDataset = pandas.concat([trainDataset.iloc[:, :1], trainDataset.iloc[:, 10:], trainDataset.iloc[:, 1:10]], axis = 1)
trainDataset.head()

### Dealing with Date_of_Journey in testing set.
##### a) Formatting the date into a datetime format.

In [ ]:
testDataset['Date_of_Journey'] = testDataset['Date_of_Journey'].astype(dtype = 'datetime64[us]')
testDataset.head()

##### b) Dividing the date in Date_of_Journey dataset into date, month and year.

In [ ]:
testDataset['Journey_start_Date'] = testDataset['Date_of_Journey'].dt.day
testDataset['Journey_start_Month'] = testDataset['Date_of_Journey'].dt.month
testDataset['Journey_start_Year'] = testDataset['Date_of_Journey'].dt.year
testDataset = testDataset.drop(columns = ['Date_of_Journey'], axis = 1)
testDataset.head()

##### c) Shifting the rows from the end to the respective position.

In [ ]:
testDataset = pandas.concat([testDataset.iloc[:, :1], testDataset.iloc[:, -3:], testDataset.iloc[:, 1:9]], axis = 1)
testDataset.head()

### Dealing with Source and Destination column.
##### In these two columns, the values from both the columns are considered as of equal importance as they hold the samilar category of data that is city values.
##### a) The unique values are selected from both the columns.
##### b) Values are combined and repeated values are removed.
##### c) Then each city is assigned a particular numerical value.
##### For e.g. : New Delhi = 1, Bangalore = 2.

In [ ]:
print(f'Unique values in train set Source : {trainDataset['Source'].unique()}')
print(f'Unique values in test set Source : {testDataset['Source'].unique()}')
print(f'Unique values in train set Destination : {trainDataset['Destination'].unique()}')
print(f'Unique values in test set Destination : {testDataset['Destination'].unique()}')

##### As, we can see there are same cities in Source and Destination for both the sets. So mapping values on one set of unique values is enough.

In [ ]:
cities = []

for columns in trainDataset['Source'].unique():
    cities.append(columns)
for columns in trainDataset['Destination'].unique():
    cities.append(columns)

cities = list(set(cities))
cities

In [ ]:
cityMap = {}
count = 1

for city in cities:
    cityMap.update({city : count})
    count += 1
    
cityMap

In [33]:
# For training set.
trainDataset['Source'] = trainDataset['Source'].map(cityMap)
trainDataset['Destination'] = trainDataset['Destination'].map(cityMap)

# For Testing Set.
testDataset['Source'] = testDataset['Source'].map(cityMap)
testDataset['Destination'] = testDataset['Destination'].map(cityMap)

In [ ]:
# Training Set.
trainDataset.head()

In [ ]:
# Testing Set.
testDataset.head()

### Dealing with the Route column in Training Set.
#### Method 1.
##### a) The city codes are extracted from the Route column.
##### b) The unique codes are then seperated to new columns and then 0 is filled in each cell.
##### c) When a particular city code is in the Route column, then 1 is assigned to that city route column at that particular index.
##### For E.G. : Route at index 0 is BLR → DEL therefore, Route_BLR = 1 and Route_DEL = 1 everything else = 0.

In [36]:
routesUniqueCodes = []

In [37]:
for index in trainDataset.index:
    route = list(trainDataset.loc[index, 'Route'].split())
    route = [r for r in route if len(r) != 1]
    routesUniqueCodes.extend(route)

routesUniqueCodes = list(set(routesUniqueCodes))

In [ ]:
len(routesUniqueCodes)

#### Method 2.
##### a) The unique Route are extracted from the Route column.
##### b) The unique Routes will be seperated to new columns and then 0 will be filled in each cell.
##### c) When a particular Route is in the Route column, then 1 will be assigned to that route column at that particular index.

In [ ]:
len(trainDataset['Route'].values)

In [ ]:
len(trainDataset['Route'].unique())

#### Choosing the Method 1, as it has less columns to work with, thus reducing high dimensions.

In [ ]:
for i in range(42):
    # Initially filling up with zero.
    trainDataset['Route_' + routesUniqueCodes[i]] = 0

trainDataset.head()

In [42]:
def update_route_map(index, routeMap):
    for city in routeMap:
        trainDataset.loc[index, 'Route_' + city] = 1

In [ ]:
for index in trainDataset.index:
    routeMap = trainDataset.loc[index, 'Route'].split()
    routeMap = [rm for rm in routeMap if len(rm) != 1]

    # Updating the route city with 1.
    update_route_map(index, routeMap)

print('Updating Done !')

#### Removing the Route column and adjusting the position of the Route Maps in the Train Set.

In [44]:
# Removing the columns.
trainDataset = trainDataset.drop(columns = ['Route'], axis = 1)

In [ ]:
# Adjusting the Route Maps.
trainDataset = pandas.concat([trainDataset.iloc[:, 0:6], trainDataset.iloc[:, 12:], trainDataset.iloc[:, 6:12]], axis = 1)
trainDataset.info()

In [ ]:
'''For the 1st route map, the value it holds is 'BLR', 'DEL'.
Therefore that column will hold the value one in that particular index,
and route map as in 'Route_BLR' and 'Route_DEL' will have value 1 at index 0.'''
trainDataset.iloc[0, 22:40]

##### As, we can see the values are updated in the particular route map in the training set.
### Dealing with the Route column in Testing Set in the same manner.

In [47]:
routesUniqueCodes = []

In [48]:
for index in testDataset.index:
    route = list(testDataset.loc[index, 'Route'].split())
    route = [r for r in route if len(r) != 1]
    routesUniqueCodes.extend(route)

routesUniqueCodes = list(set(routesUniqueCodes))

In [ ]:
len(routesUniqueCodes)

In [ ]:
for i in range(38):
    # Initially filling up with zero.
    testDataset['Route_' + routesUniqueCodes[i]] = 0

testDataset.head()

In [51]:
def update_route_map(index, routeMap):
    for city in routeMap:
        testDataset.loc[index, 'Route_' + city] = 1

In [ ]:
for index in testDataset.index:
    routeMap = testDataset.loc[index, 'Route'].split()
    routeMap = [rm for rm in routeMap if len(rm) != 1]

    # Updating the route city with 1.
    update_route_map(index, routeMap)

print('Updating Done !')

#### Removing the Route column and adjusting the position of the Route Maps in the Test Set.

In [53]:
# Removing the columns.
testDataset = testDataset.drop(columns = ['Route'], axis = 1)

In [ ]:
# Adjusting the Route Maps.
testDataset = pandas.concat([testDataset.iloc[:, 0:6], testDataset.iloc[:, 11:], testDataset.iloc[:, 6:11]], axis = 1)
testDataset.info()

In [ ]:
testDataset.iloc[0, 19:35]

##### The values are updated in the particular Route Map in the testing set.
### Dealing with the Dep_Time Column.
##### Dividing the arrival time into hours and minutes columns.
#### Training Set.

In [ ]:
for index in trainDataset.index:
    trainDataset.loc[index, 'Dep_Time_Hour'] = trainDataset.loc[index, 'Dep_Time'].split(':')[0]
    trainDataset.loc[index, 'Dep_Time_Minutes'] = trainDataset.loc[index, 'Dep_Time'].split(':')[1]

trainDataset.head()

In [57]:
trainDataset = trainDataset.drop(columns = ['Dep_Time'], axis = 1)

In [ ]:
trainDataset = pandas.concat([trainDataset.iloc[:, :48], trainDataset.iloc[:, -2:], trainDataset.iloc[:, 48:53]], axis = 1)
trainDataset.info()

#### Testing Set.

In [ ]:
for index in testDataset.index:
    testDataset.loc[index, 'Dep_Time_Hour'] = testDataset.loc[index, 'Dep_Time'].split(':')[0]
    testDataset.loc[index, 'Dep_Time_Minutes'] = testDataset.loc[index, 'Dep_Time'].split(':')[1]

testDataset.head()

In [60]:
testDataset = testDataset.drop(columns = ['Dep_Time'], axis = 1)

In [ ]:
testDataset = pandas.concat([testDataset.iloc[:, :44], testDataset.iloc[:, -2:], testDataset.iloc[:, 44:48]], axis = 1)
testDataset.info()

### Dealing with the Arrival_Time in the same manner.
#### Training Set.

In [ ]:
trainDataset['Arrival_Time'].unique()

In [ ]:
print(trainDataset['Arrival_Time'].unique())
print(len(trainDataset['Arrival_Time'].unique()))

##### Since, the arrival time has dates along with time dividing both the values into seperate columns.

In [ ]:
for index in trainDataset.index:
    trainDataset.loc[index, 'Arrival_Time_Hour'] = trainDataset.loc[index, 'Arrival_Time'].split(':')[0]
    
    if len(trainDataset.loc[index, 'Arrival_Time'].split(':')[1]) > 2:
        trainDataset.loc[index, 'Arrival_Time_Minutes'] = trainDataset.loc[index, 'Arrival_Time'].split(':')[1][0:2:1]

        trainDataset.loc[index, 'Journey_End'] = trainDataset.loc[index, 'Arrival_Time'].split(':')[1][2::]
    else:
        trainDataset.loc[index, 'Arrival_Time_Minutes'] = trainDataset.loc[index, 'Arrival_Time'].split(':')[1]

        trainDataset.loc[index, 'Journey_End'] = '-'

trainDataset.head()

In [ ]:
for index in trainDataset.index:
    if trainDataset.loc[index, 'Journey_End'] != '-':
        trainDataset.loc[index, 'Journey_End_Date'] = trainDataset.loc[index, 'Journey_End'].split()[0]
        trainDataset.loc[index, 'Journey_End_Month'] = trainDataset.loc[index, 'Journey_End'].split()[1]
    else:
        trainDataset.loc[index, 'Journey_End_Date'] = 0
        trainDataset.loc[index, 'Journey_End_Month'] = '-'

trainDataset['Journey_End_Month'].unique()

##### Mapping the months to the respective month number.

In [66]:
trainDataset['Journey_End_Month'] = trainDataset['Journey_End_Month'].map({'Mar' : 3, 'Jun' : 6, 'May' : 5, 'Apr' : 4, '-' : 0})

In [67]:
trainDataset = trainDataset.drop(columns = ['Arrival_Time', 'Journey_End'], axis = 1)

In [ ]:
trainDataset = pandas.concat([trainDataset.iloc[:, :50], trainDataset.iloc[:, -4:], trainDataset.iloc[:, 50:54]], axis = 1)
trainDataset.info()

#### Testing Set.

In [ ]:
testDataset['Arrival_Time'].unique()[1:10]

In [ ]:
for index in testDataset.index:
    testDataset.loc[index, 'Arrival_Time_Hour'] = testDataset.loc[index, 'Arrival_Time'].split(':')[0]

    if len(testDataset.loc[index, 'Arrival_Time'].split(':')[1]) > 2:
        testDataset.loc[index, 'Arrival_Time_Minutes'] = testDataset.loc[index, 'Arrival_Time'].split(':')[1][0:2:1]
        testDataset.loc[index, 'Journey_End'] = testDataset.loc[index, 'Arrival_Time'].split(':')[1][2::]

    else:
        testDataset.loc[index, 'Arrival_Time_Minutes'] = testDataset.loc[index, 'Arrival_Time'].split(':')[1]
        testDataset.loc[index, 'Journey_End'] = '-'

testDataset.head()

In [ ]:
for index in testDataset.index:
    if testDataset.loc[index, 'Journey_End'] != '-':
        testDataset.loc[index, 'Journey_End_Date'] = testDataset.loc[index, 'Journey_End'].split()[0]
        testDataset.loc[index, 'Journey_End_Month'] = testDataset.loc[index, 'Journey_End'].split()[1]
    else:
        testDataset.loc[index, 'Journey_End_Date'] = 0
        testDataset.loc[index, 'Journey_End_Month'] = '-'

testDataset['Journey_End_Month'].unique()

In [72]:
testDataset['Journey_End_Month'] = testDataset['Journey_End_Month'].map({'Jun' : 6, 'May' : 5, 'Mar' : 3, 'Apr' : 4, '-' : 0})

In [73]:
testDataset = testDataset.drop(columns = ['Arrival_Time', 'Journey_End'], axis = 1)

In [ ]:
testDataset.info()

In [ ]:
testDataset = pandas.concat([testDataset.iloc[:, :46], testDataset.iloc[:, -4:], testDataset.iloc[:, 46:49]], axis = 1)
testDataset.info()

### Dealing with Duration.
##### In this column the hours and minutes are converted into minutes, for easier calculation.
#### Training Set.

In [ ]:
for index in trainDataset.index:
    duration = trainDataset.loc[index, 'Duration'].split()

    if len(duration) == 2:
        trainDataset.loc[index, 'Duration'] = int(duration[0][0:len(duration[0]) - 1:1])*60 + int(duration[1][0:len(duration[1]) - 1: 1])
        

    if len(duration) < 2:
        if duration[0][-1:-2:-1] == 'h' or duration[0][- 1:-2:-1] == 'H':
            trainDataset.loc[index, 'Duration'] = int(duration[0][0:len(duration[0]) - 1:1])*60
        

        if duration[0][-1:-2:-1] == 'm' or duration[0][-1:-2:-1] == 'M':
            trainDataset.loc[index, 'Duration'] = int(duration[0][0:len(duration[0]) - 1:1])

del duration
trainDataset['Duration'].head()

#### Testing Set.

In [ ]:
for index in testDataset.index:
    duration = testDataset.loc[index, 'Duration'].split()

    if len(duration) == 2:
        testDataset.loc[index, 'Duration'] = int(duration[0][0:len(duration[0]) - 1:1])*60 + int(duration[1][0:len(duration[1]) - 1: 1])
        

    if len(duration) < 2:
        if duration[0][-1:-2:-1] == 'h' or duration[0][- 1:-2:-1] == 'H':
            testDataset.loc[index, 'Duration'] = int(duration[0][0:len(duration[0]) - 1:1])*60
        

        if duration[0][-1:-2:-1] == 'm' or duration[0][-1:-2:-1] == 'M':
            testDataset.loc[index, 'Duration'] = int(duration[0][0:len(duration[0]) - 1:1])

del duration
testDataset['Duration'].head()

### Dealing with the Total_Stops Column.
##### Mapping the Total_Stops Columns as the number of stops between Source and Destination. For e.g.: 4 stops will be considered as 4. 
#### Training Set

In [ ]:
trainDataset['Total_Stops'].unique()

In [ ]:
trainDataset['Total_Stops'] = trainDataset['Total_Stops'].map({'non-stop' : 0, '2 stops' : 2, '1 stop' : 1, '3 stops' : 3, '4 stops' : 4})
trainDataset['Total_Stops'].head()

#### Testing Set.

In [ ]:
testDataset['Total_Stops'].unique()

In [ ]:
testDataset['Total_Stops'] = testDataset['Total_Stops'].map({'non-stop' : 0, '2 stops' : 2, '1 stop' : 1, '3 stops' : 3, '4 stops' : 4})
testDataset['Total_Stops'].head()

### Dealing with the Additional_Info column.
##### Mapping the values with integer values.
#### Training Set.

In [ ]:
trainDataset['Additional_Info'].unique()

##### Changing the 'No info' and and 'No Info' as 'No-info'.

In [83]:
for index in trainDataset.index:
    if trainDataset.loc[index, 'Additional_Info'] == 'No info':
        trainDataset.loc[index, 'Additional_Info'] = 'No-info'
    if trainDataset.loc[index, 'Additional_Info'] == 'No Info':
        trainDataset.loc[index, 'Additional_Info'] = 'No-info'

In [ ]:
add_info = {}
count = 0

for values in trainDataset['Additional_Info'].unique():
    add_info.update({values : count})
    count += 1

add_info

In [ ]:
trainDataset['Additional_Info'] = trainDataset['Additional_Info'].map(add_info)
trainDataset['Additional_Info'].head()

In [ ]:
trainDataset['Additional_Info'].unique()

#### Testing Set.

In [ ]:
testDataset['Additional_Info'].unique()

In [ ]:
add_info = {}
count = 0

for values in testDataset['Additional_Info'].unique():
    add_info.update({values : count})
    count += 1

testDataset['Additional_Info'] = testDataset['Additional_Info'].map(add_info)
testDataset['Additional_Info'].unique()

In [ ]:
trainDataset.head()

In [ ]:
testDataset.head()

### Changing the data types of the columns.

In [ ]:
# For training Set.
for column in trainDataset.columns:
    if trainDataset[column].dtype.name == 'object':
        trainDataset[column] = trainDataset[column].astype('int64')

# For testing Set.
for column in testDataset.columns:
    if testDataset[column].dtype.name == 'object':
        testDataset[column] = testDataset[column].astype('int64')

trainDataset.info()

In [ ]:
testDataset.info()

# Final Dataset.{-}

In [ ]:
trainDataset.head()

In [ ]:
testDataset.head()

# Converting them to CSV file.

In [ ]:
trainDataset.to_csv('Cleaned Training Set.csv')
testDataset.to_csv('Cleaned Testing Set.csv')

print('File Saved !')